In [0]:
%%sh
PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF  
--select * from market.market_adapp_fact_v1 limit 1;

--explain select creative_count,ad_platform_id from market.market_adapp_fact_v1 where granularity='monthly' and device_code = 'ios-phone' and  country_code = 'WW' and date='2020-03-31' and ad_app_id=510461758  and ad_platform_id=1;
--select * from  market.market_creative_adapp_fact_v1 where granularity='monthly' and  device_code = 'ios-phone' and country_code in ('DE', 'TW', 'ID', 'BR', 'RU', 'CA', 'DK', 'ES', 'HK', 'US', 'NO', 'FR', 'KR', 'TH', 'JP', 'GB', 'IT', 'NZ', 'TR', 'AU', 'MX', 'SE', 'CN', 'IN', 'FI') and date='2020-03-31' and ad_app_id=510461758 limit 1;

-- select ad_app_id from market.market_adapp_fact_v1 where  country_code = 'US' and device_code = 'ios-phone' and date='2019-03-31'  limit 100;




-- select creative_count from market.market_adapp_fact_v1 where ad_app_id=510461758 and country_code = 'JP' and device_code = 'ios-phone' and date='2019-03-31' and ad_platform_id=1  limit 3;
-- select count(distinct(creative_id)) from  market.market_creative_adapp_fact_v1 where ad_app_id=510461758 and country_code = 'JP' and device_code = 'ios-phone' and date='2019-03-31' limit 10;



-- sum(creative_count) from market.market_adapp_fact_v1 where 
            granularity='weekly' AND device_code = 'ios-phone' AND  
            country_code = 'DE' AND date='2020-04-11' AND ad_app_id=1208907397  AND ad_platform_id =1;

select creative_count from market.market_adapp_fact_v1 where 
            granularity='monthly' AND device_code = 'ios-phone' AND  
            country_code = 'BR' AND date='2020-03-31' AND ad_app_id=284035177 and ad_platform_id=325;

select count(distinct(creative_id)) from  market.market_creative_adapp_fact_v1 where 
            granularity='monthly' AND  device_code = 'ios-phone' AND country_code ='BR'
            AND date='2020-03-31' AND ad_app_id=284035177 and ad_platform_id=325;
            
            


-- 1208907397

-- select creative_count from market.market_adapp_fact_v1 where ad_app_id=1208907397 and country_code = 'DE' and device_code = 'ios-phone' and date='2020-04-04' and ad_platform_id=101;
-- select count(distinct(creative_id)) from  market.market_creative_adapp_fact_v1 where ad_app_id=1208907397 and country_code = 'DE' and device_code = 'ios-phone' and date='2020-04-04' and ad_platform_id=101 AND (
                        ad_platform_count IS NOT NULL
                        or ad_platform_list IS NOT NULL
                        or pub_app_count IS NOT NULL
                        or pub_app_list IS NOT NULL
                        or country_count IS NOT NULL
                        or country_list IS NOT NULL
                        or creative_share_of_ad_platform IS NOT NULL
                        or creative_share_of_advertiser IS NOT NULL
                    );
                    
--select count(distinct creative_id) from market. market_creative_log_fact_v1 where device_code='ios-phone' and country_code = 'DE' and ad_app_id=1208907397 and date between '2020-03-29' and '2020-04-04' and ad_platform_id=101;



--select * from market.market_creative_map_dimension_v1 where creative_id in ()




EOF

In [0]:
%%sh
PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF  
select * 
from   market.market_creative_adapp_fact_v1 
where  ad_app_id = 1208907397 
       and creative_id not in (select distinct( creative_id ) 
                               from   market.market_creative_log_fact_v1 
                               where  ad_app_id = 1208907397 
                                      and country_code = 'DE' 
                                      and device_code = 'ios-phone' 
                                      and date between '2020-03-29' and '2020-04-04' 
                                      and ad_platform_id = 101) 
       and country_code = 'DE' 
       and device_code = 'ios-phone' 
       and date = '2020-04-04' 
       and ad_platform_id = 101 
       





EOF

In [0]:

spark.read.parquet("s3://prod.appannie.mktint.data/market_fact/granularity=WEEK/date=2020-04-04").cache().createOrReplaceTempView("source")


In [0]:

spark.sql("""
select add_app_id, creative_id,creative_SOV,creative_adapp_SOV
from source platform_id = 2 and device_id=1 and country_code='DE' and ad_app_id=1208907397 and cretive_id in (90100014246392, 90100014246432)
where 
""").show()

In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
from datetime import datetime, timedelta
import random
import time

import pandas as pd
CITUS_MKT_HOSTS = [('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com', 7432)]
CITUS_MKT_NAME = 'aa_mkt_db'
CITUS_MKT_ACCESS_ID = 'citus_mkt_qa'
CITUS_MKT_SECRET_KEY = 'rn*Wh%osCl2C'
CITUS_MKT_CONNECTION_CAPACITY = 10
CITUS_MKT_SEMAPHORE_TIMEOUT = 7201

mkt_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result

COUNTRY_CODE_BASIC = sorted(["AU", "BR", "CA", "CN", "DE", "DK", "ES", "FI", "FR", "GB", "HK", "ID", "IN", "IT", "JP",
                             "KR", "MX", "NO", "NZ", "RU", "SE", "TH", "TR", "TW", "US"])


start = time.time()
test = """select count(distinct(creative_id)),date,granularity from  market.market_creative_adapp_fact_v1 where  device_code = 'ios-phone' and country_code in ('DE', 'TW', 'ID', 'BR', 'RU', 'CA', 'DK', 'ES', 'HK', 'US', 'NO', 'FR', 'KR', 'TH', 'JP', 'GB', 'IT', 'NZ', 'TR', 'WW', 'AU', 'MX', 'SE', 'CN', 'IN', 'FI') and ad_app_id=585193266 group by date,granularity;"""

p = """select count(1) from market.market_creative_adapp_fact_v1 where 
                 True  AND 
                 pub_app_count is not null AND pub_app_list is null;"""
q = """select count(1) from market.market_creative_adapp_fact_v1 where 
                 pub_app_count is not null AND pub_app_list is null;
"""
r = query(mkt_dsn, q)
m = query(mkt_dsn, p)
end = time.time()
print end-start
print r
print "################"
print m


In [0]:

import os
import multiprocessing
import threading
from multiprocessing.dummy import Pool

# select count(1) from market.market_creative_adapp_fact_v1 where granularity='weekly' and device_code='ios-phone' and country_code='US' and pub_app_count is not null and pub_app_list is null
# granularity = ["weekly", "monthly"]
# device_code = ["android-phone", "android-tablet", "ios-phone", "ios-tablet"]
country_code = ['DE', 'TW', 'ID', 'BR', 'RU', 'CA', 'DK', 'ES', 'HK', 'US', 'NO', 'FR', 'KR', 'TH', 'JP', 'GB', 'IT', 'NZ', 'TR', 'WW', 'AU', 'MX', 'SE', 'CN', 'IN', 'FI']
granularity = ["weekly", "monthly"]
device_code = ["android-phone", "android-tablet", "ios-phone", "ios-tablet"]
# country_code = ['HK', 'US']
start = time.time()



def count_all(country_code):
    r = []
    for _g in granularity:
        for _d in device_code:
            
                sql = """select count(1) from market.market_creative_adapp_fact_v1 where granularity='{}' and device_code='{}' and country_code='{}' and pub_app_count is not null and pub_app_list is null;""".format(_g,_d,country_code)
                res = query(mkt_dsn, sql)
                assert res[0][0] == 0
                # print threading.current_thread().ident, multiprocessing.current_process().pid
            

start = time.time()
pool = multiprocessing.Pool(multiprocessing.cpu_count())
pool.map(count_all, country_code)
end = time.time()
print end-start




 















In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
from datetime import datetime, timedelta
import random
import time

import pandas as pd
CITUS_MKT_HOSTS = [('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com', 7432)]
CITUS_MKT_NAME = 'aa_mkt_db'
CITUS_MKT_ACCESS_ID = 'citus_mkt_qa'
CITUS_MKT_SECRET_KEY = 'rn*Wh%osCl2C'
CITUS_MKT_CONNECTION_CAPACITY = 10
CITUS_MKT_SEMAPHORE_TIMEOUT = 7201

mkt_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)

urn = Urn(
    namespace="app-qa.db-check.v1",
    owner="app_qa"
)
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
    
    


start = time.time()
q = '''select count(1) from market.market_creative_adapp_fact_v1 where 
             country_count != array_length(regexp_split_to_array(country_list, ','),1) 
             and country_count is not null and country_list is not null and country_code = 'WW';'''

p = """select count(1) from market.market_creative_adapp_fact_v1 where country_count != array_length(regexp_split_to_array(country_list, ','),1) AND country_count is not null AND country_list is not null AND country_code = 'WW';"""
r_1 = query(mkt_dsn, q)
end = time.time()
print end-start
print r_1


In [0]:

from aadatapipelinecore.core.urn import Urn
import psycopg2
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
from aaplproxy.connection import ClusterConnection
from conf import settings
from aadatapipelinecore.core.loader.plproxy import build_db_settings
import datetime
from datetime import timedelta
from applications.db_check_v1.common.utils import string_to_datetime
from applications.db_check_v1.common.html_report_test_runner import HTMLTestRunner
import random
import croniter

import sys
import datetime
import traceback
import unittest
SUPPORTED_DEVICE_CODE = sorted(["ios-phone", "ios-tablet", "android-phone", "android-tablet"])
COUNTRY_CODE_WW = ["WW"]

CITUS_MKT_HOSTS = [('internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com', 7432)]
CITUS_MKT_NAME = 'aa_mkt_db'
CITUS_MKT_ACCESS_ID = 'citus_mkt_qa'
CITUS_MKT_SECRET_KEY = 'rn*Wh%osCl2C'
CITUS_MKT_CONNECTION_CAPACITY = 10
CITUS_MKT_SEMAPHORE_TIMEOUT = 7201
AD_APP_ID =[1412159494, 1258027083, 585193266, 1104911270, 1356481770, 630687854, 821068605, 594237344, 1114471030,
             1336356897, 1374974523, 695061867, 1347684152, 1389003044, 1454573587, 945164641, 1243351872, 1171717002,
             454607051, 1065781769, 1436001994, 1341461714, 1404798531, 1447221122, 1111570198, 990704801, 1126877157,
             1189828120, 1403912708, 1412702666, 975840117, 1013992870, 1214783848, 1113619083, 1328359160, 1248364395,
             1419779267, 1431652705, 1091456207, 1277029359, 1449170096, 1441208516, 542557212, 1446696024, 525373618,
             1223937103, 1006386292, 1289636462, 1447871511, 1438413584, 486781045, 1208907397, 694133630, 1339105679,
             1438413584, 1277029359, 1208907397, 1208907397, 1216135390, 1438413584, 1450753813, 980067261, 1339105679,
             1208907397, 1450753813, 1257031979, 1182397101, 1447871511, 1447871511, 1263426717, 1438413584, 1277029359,
             1450753813, 1223397201, 1006386292, 792750948, 1216135390, 1447871511, 1103438575, 427916203, 1263426717,
             1073286211, 1091456207, 1091456207, 1277029359, 1065781769, 1223937103, 1093860285, 922103212, 1167152768,
             1347684152, 922103212, 1247414258, 1233777304, 1436327767, 1447871511, 789844385, 1208907397, 1223397201,
             1450753813, 694133630, 454607051, 1065781769, 1126877157, 694133630, 1208907397, 1093860285, 792750948,
             1103438575, 1171717002, 1257031979, 1247414258, 792750948, 1103438575, 427916203, 1289636462, 1431652705,
             486781045, 1447871511, 792750948, 1336356897, 1182397101, 980067261, 1091131962, 922103212, 1438413584,
             1454573587, 1006386292, 922103212, 1258027083, 1247414258, 945164641, 1167152768, 1356481770, 1214783848,
             1404798531, 1446696024, 454607051, 486781045, 1091456207, 1258027083, 427916203, 1277029359, 1104911270]
AD_PLATFORM_ID_BASIC = sorted([203, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 116, 142, 311, 121])
COUNTRY_CODE_BASIC = sorted(["AU", "BR", "CA", "CN", "DE", "DK", "ES", "FI", "FR", "GB", "HK", "ID", "IN", "IT", "JP",
                             "KR", "MX", "NO", "NZ", "RU", "SE", "TH", "TR", "TW", "US"])
AD_PLATFORM_ID_ALL_PLATFORM = [1]
AD_PLATFORM_ID_BASIC = sorted([203, 101, 136, 201, 202, 171, 109, 110, 111, 112, 200, 116, 142, 311, 121])
AD_PLATFORM_ID_CREATIVE_ONLY = [324, 325]
SUPPORTED_DIMENSIONS = {
    "ad_platform_id": sorted(AD_PLATFORM_ID_BASIC + AD_PLATFORM_ID_ALL_PLATFORM + AD_PLATFORM_ID_CREATIVE_ONLY),
    "ad_platform_id_non_creative_only": sorted(AD_PLATFORM_ID_BASIC + AD_PLATFORM_ID_ALL_PLATFORM),
    "ad_platform_id_non_creative_only_non_all_platform": sorted(AD_PLATFORM_ID_BASIC),
    "device_code": sorted(SUPPORTED_DEVICE_CODE),
    "country_code": sorted(COUNTRY_CODE_BASIC + COUNTRY_CODE_WW),
    "country_code_non_ww": sorted(COUNTRY_CODE_BASIC),
}


def _get_date_from_refresh_routing_config(config):
   
    schedule, days_delta = config
    # here use UTC now
    cron = croniter.croniter(schedule, datetime.datetime.utcnow())
    date = cron.get_prev(datetime.datetime) - datetime.timedelta(days=days_delta)
    return date.strftime("%Y-%m-%d")
citus_dsn = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db=CITUS_MKT_NAME,
        user=CITUS_MKT_ACCESS_ID,
        host=CITUS_MKT_HOSTS[0][0],
        password=CITUS_MKT_SECRET_KEY,
        port=CITUS_MKT_HOSTS[0][1]
    )
)
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
class TestMarketCompletenessBase(unittest.TestCase):
    check_date = None

    def setUp(self):
        self.granularity = None

    def market_dimension_v1(self):
        tables = [
            "market_creative_map_dimension_v1",
            "market_campaign_map_dimension_v1",
            "market_creative_meta_dimension_v1"
        ]
        for table in tables:
            sql = "SELECT COUNT(*) FROM market.{table} WHERE update_date='{date}'".format(date=self.check_date,
                                                                                          table=table)
            result = query(citus_dsn, sql)
            count = result[0][0]
            assert count > 0

    def market_creative_log_fact_v1(self):
        table = "market_creative_log_fact_v1"
        sql_filter = {"date": self.check_date}
        self._completeness_check_on_dimension(table, "country_code", sql_filter)
        self._completeness_check_on_dimension(table, "device_code", sql_filter)
        self._completeness_check_on_dimension(table, "ad_platform_id", sql_filter)

    def market_metric_facts_v1(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_pubapp_fact_v1",
            "market_adapp_fact_v1",
            "market_adapp_publisher_fact_v1",
            "market_pubapp_fact_v1",
        ]
        sql_filter = {"date": self.check_date, "granularity": self.granularity}
        for table in tables:
            if table in ["market_adapp_publisher_fact_v1"]:
                self._completeness_check_on_dimension(table, "country_code", sql_filter,
                                                      SUPPORTED_DIMENSIONS["country_code_non_ww"])
                self._completeness_check_on_dimension(table, "ad_platform_id", sql_filter,
                                                      SUPPORTED_DIMENSIONS[
                                                          "ad_platform_id_non_creative_only_non_all_platform"])
            elif table in ["market_pubapp_fact_v1", "market_adapp_pubapp_fact_v1"]:
                self._completeness_check_on_dimension(table, "country_code", sql_filter)
                self._completeness_check_on_dimension(table, "ad_platform_id", sql_filter,
                                                      SUPPORTED_DIMENSIONS["ad_platform_id_non_creative_only"])
            elif table in ["market_creative_adapp_fact_v2", "market_adapp_fact_v1"]:
                self._completeness_check_on_dimension(table, "ad_platform_id", sql_filter)
                self._completeness_check_on_dimension(table, "country_code", sql_filter)
            self._completeness_check_on_dimension(table, "device_code", sql_filter)

    def market_rank_facts_v1(self):
        tables = [
            "market_creative_cat_rank_fact_v1",
            "market_adapp_cat_rank_fact_v1",
            "market_pubapp_cat_rank_fact_v1",
        ]
        sql_filter = {"date": self.check_date, "granularity": self.granularity}
        for table in tables:
            self._completeness_check_on_dimension(table, "country_code", sql_filter)
            self._completeness_check_on_dimension(table, "device_code", sql_filter)
            # _completeness_check_on_dimension(table, "category_id", sql_filter)

    def market_seen_facts_v1(self):
        tables = [
            "market_creative_adapp_seen_fact_v1",
            "market_adapp_pubapp_seen_fact_v1"
        ]
        # here will not checking "first_seen_date", check with DF, as there may no new creative
        seens = ["last_seen_date"]
        for table in tables:
            for seen in seens:
                sql_filter = {seen: self.check_date}
                self._completeness_check_on_dimension(table, "country_code", sql_filter)
                self._completeness_check_on_dimension(table, "device_code", sql_filter)
                self._completeness_check_on_dimension(table, "ad_platform_id", sql_filter)

    def _completeness_check_on_dimension(self, table, dimension, sql_filter, expected_dimensions=None):
        condition = " AND ".join(["{}='{}'".format(key, value) for key, value in sql_filter.iteritems()]) \
            if sql_filter else "True"
        sql = "SELECT DISTINCT({dimension}) FROM market.{table} WHERE {condition}".format(
            dimension=dimension, table=table, condition=condition)
        result = query(citus_dsn, sql)
        actual_list = sorted([obj[0] for obj in result])
        expected_dimensions = SUPPORTED_DIMENSIONS[dimension] if expected_dimensions is None else expected_dimensions

        # TODO ad_platform_id = 316 missed in db from 2019-09, and it is a known issue @ 2019-10-20
        if 'date' in sql_filter and sql_filter['date'] == '2019-09-30' and dimension == "ad_platform_id":
            expected_dimensions = sorted(expected_dimensions + [316])
        if expected_dimensions is None:
            raise RuntimeError("expected_dimensions is None")
        if actual_list != expected_dimensions:
            error_msg = "dimensions miss matched on {} table where {} : expected {} - actual {}. ".format(
                table, condition, expected_dimensions, actual_list)
            raise RuntimeError(error_msg)

    def market_seen_facts_v1_test(self):
        tables = [
            "market_creative_adapp_seen_fact_v1",
            "market_adapp_pubapp_seen_fact_v1"
        ]
        seens = ["first_seen_date", "last_seen_date"]
        for table in tables:
            for seen in seens:
                sql_filter = {seen: self.check_date}
                self._completeness_check_on_dimension(table, "country_code", sql_filter)
                self._completeness_check_on_dimension(table, "device_code", sql_filter)
                self._completeness_check_on_dimension(table, "ad_platform_id", sql_filter)


class TestMarketDimensionDaily(TestMarketCompletenessBase):
    # dimension check, the update_date is the db load date

    trigger_date_config = ("0 9 * * *", 0)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "daily"

    def test_market_dimension_v1_daily(self):
        self.market_dimension_v1()


class TestMarketLogsFactAndSeenDaily(TestMarketCompletenessBase):
    # logs fact and seen check, the date should be 1 days ago
    trigger_date_config = ("0 9 * * *", 1)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "daily"

    def test_market_creative_log_fact_v1_daily(self):
        self.market_creative_log_fact_v1()

    def test_market_seen_facts_v1_daily(self):
        self.market_seen_facts_v1()


class TestMarketWeeklyCheck(TestMarketCompletenessBase):
    trigger_date_config = ("0 10 * * 0", 8)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "weekly"

    def test_market_metric_facts_v1_weekly(self):
        self.market_metric_facts_v1()

    def test_market_rank_facts_v1_weekly(self):
        self.market_rank_facts_v1()


class TestMarketMonthlyCheck(TestMarketCompletenessBase):
    trigger_date_config = ("0 10 6 * * ", 6)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = "monthly"

    def test_market_metric_facts_v1_monthly(self):
        self.market_metric_facts_v1()

    def test_market_rank_facts_v1_monthly(self):
        self.market_rank_facts_v1()
class TestMarketCountIssueRegressionBase(unittest.TestCase):
    check_date = None

    def setUp(self):
        self.granularity = None
        self.is_routine_test = True
        self.ad_app_ids = None

    def _compose_condition(self, sql, table):
        if not self.is_routine_test:
            return sql.format(table=table, date_granularity_condition=True)
        return sql.format(table=table,
                          date_granularity_condition="granularity='{}' AND date='{}'".format(self.granularity,
                                                                                             self.check_date))

    def market_pub_app_count_none(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                 {date_granularity_condition}  AND 
                 pub_app_count is not null AND pub_app_list is null;"""

        for table in tables:
            sql = self._compose_condition(sql, table)
            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result,
                             msg="The num of (pub_count is none/pub_list is not none) is {} in table {}.".format(
                                 result, table))

    def market_pub_app_count_not_none(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                    {date_granularity_condition} AND 
                    pub_app_count is null AND pub_app_list is not null;"""
        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result,
                             msg="The num of (pub_count is not none/pub_list is none) is {} in table {}.".format(
                                 result, table))

    def market_pub_app_count_list_equal(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where {date_granularity_condition} AND
                         pub_app_count != array_length(regexp_split_to_array(pub_app_list, ','),1) 
                         AND array_length(regexp_split_to_array(pub_app_list, ','),1) < 20
                         AND pub_app_count is not null AND pub_app_list is not null;"""

        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result, msg="The num of (pub_count doesn't equal to the length of pub_list) is {} in "
                                         "table {}.".format(result, table))

    def market_ad_platform_count_none(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1",
            "market_adapp_pubapp_fact_v1",
            "market_pubapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                    {date_granularity_condition} AND 
                    ad_platform_count is not null AND ad_platform_list is null;"""

        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result, msg="The num of (ad_platform_count is none/ad_platform_list is not none) is {} in "
                                         "table {}.".format(result, table))

    def market_ad_platform_count_not_none(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1",
            "market_adapp_pubapp_fact_v1",
            "market_pubapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                    {date_granularity_condition} AND 
                    ad_platform_count is null AND ad_platform_list is not null;"""

        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result, msg="The num of (ad_platform_count is none/ad_platform_list is not none) is {} in "
                                         "table {}.".format(result, table))

    def market_ad_platform_count_list_equal(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1",
            "market_adapp_pubapp_fact_v1",
            "market_pubapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                    {date_granularity_condition} AND 
                    ad_platform_count != array_length(regexp_split_to_array(ad_platform_list, ','),1) AND
                     ad_platform_count is not null AND ad_platform_list is not null AND ad_platform_id = 1;"""

        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result,
                             msg="The num of (ad_platform_count doesn't equal to the length of ad_platform_list) "
                                 "is {} in table {}.".format(result, table))

    def market_not_all_ad_platform_count_list_none(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1",
            "market_adapp_pubapp_fact_v1",
            "market_pubapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                    {date_granularity_condition} AND 
                    (ad_platform_count is not null or ad_platform_list is not null) AND 
                    ad_platform_id != 1;"""

        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result, msg="The num of (ad_platform_count/list is not null for other platforms except "
                                         "All network) is {} in table {}.".format(result, table))

    def market_country_count_none(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                    {date_granularity_condition} AND 
                    country_count is not null AND country_list is null;"""

        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result, msg="The num of (country_count is none/country_list is not none) is {} in "
                                         "table {}.".format(result, table))

    def market_country_count_not_none(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                    {date_granularity_condition} AND 
                    country_count is null AND country_list is not null;"""

        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result, msg="The num of (country_count is not none/country_list is none) is {} in "
                                         "table {}.".format(result, table))

    def market_country_count_list_equal(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                    {date_granularity_condition} AND 
                    country_count != array_length(regexp_split_to_array(country_list, ','),1) AND 
                    country_count is not null AND country_list is not null AND country_code = 'WW';"""

        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result,
                             msg="The num of (country_count doesn't equal to the length of country_list) is {} in "
                                 "table {}.".format(result, table))

    def market_country_count_list_not_ww(self):
        tables = [
            "market_creative_adapp_fact_v2",
            "market_adapp_fact_v1"
        ]
        sql = """select count(1) from market.{table} where 
                    {date_granularity_condition} AND 
                    (country_count is not null or country_list is not null) AND 
                    country_code != 'WW';"""

        for table in tables:
            sql = self._compose_condition(sql, table)

            result = query(citus_dsn, sql)[0][0]
            self.assertFalse(result,
                             msg="The num of (country_count/list is not null for other countries except WW) is {} "
                                 "in table {}.".format(result, table))

    def market_creative_count_equal_advertiser_report_creative(self):
        for ad_app_id in self.ad_app_ids:
            sql_1 = """select creative_count from market.market_adapp_fact_v1 where 
            granularity='{granularity}' AND device_code = 'ios-phone' AND  
            country_code = 'WW' AND date='{date}' AND ad_app_id={ad_app_id}  AND ad_platform_id=1;""".format(
                granularity=self.granularity, date=self.check_date, ad_app_id=ad_app_id)

            sql_2 = """select count(distinct(creative_id)) from  market.market_creative_adapp_fact_v2 where 
            granularity='{granularity}' AND  device_code = 'ios-phone' AND country_code in ({country_code}) 
            AND date='{date}' AND ad_app_id={ad_app_id};""".format(granularity=self.granularity,
                                                                   country_code=str(COUNTRY_CODE_BASIC)[1:-1],
                                                                   date=self.check_date,
                                                                   ad_app_id=ad_app_id)
            result_1 = query(citus_dsn, sql_1)
            result_2 = query(citus_dsn, sql_2)[0][0]
            if len(result_1) > 0:
                self.assertEqual(result_1[0][0], result_2,
                                 msg="In tabele :'market_adapp_fact_v1' creative count of {} is not equal creative "
                                     "count in "
                                     "table: 'market_creative_adapp_fact_v2'.".format(ad_app_id))
            else:
                self.assertEqual(result_2, 0,
                                 msg="In table 'market_creative_adapp_fact_v2': {}, {}, {} creative count not equal"
                                 .format(
                                     self.check_date, self.granularity,
                                     ad_app_id))


class TestMarketCountIssueWeeklyRegression(TestMarketCountIssueRegressionBase):
    trigger_date_config = ("0 10 * * 0", 8)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = None
        self.is_routine_test = True
        self.ad_app_ids = random.sample(AD_APP_ID, 100)

    def test_pub_app_count_none(self):
        self.market_pub_app_count_none()

    def test_pub_app_count_not_none(self):
        self.market_pub_app_count_not_none()

    def test_pub_app_count_list_equal(self):
        self.market_pub_app_count_list_equal()

    def test_ad_platform_count_none(self):
        self.market_ad_platform_count_none()

    def test_ad_platform_count_not_none(self):
        self.market_ad_platform_count_not_none()

    def test_ad_platform_count_list_equal(self):
        self.market_ad_platform_count_list_equal()

    def test_not_all_ad_platform_count_list_none(self):
        self.market_not_all_ad_platform_count_list_none()

    def test_country_count_none(self):
        self.market_country_count_none()

    def test_country_count_not_none(self):
        self.market_country_count_not_none()

    def test_country_count_list_equal(self):
        self.market_country_count_list_equal()

    def test_country_count_list_not_ww(self):
        self.market_country_count_list_not_ww()

    def test_creative_count(self):
        self.market_creative_count_equal_advertiser_report_creative()


class TestMarketCountIssueMonthlyRegression(TestMarketCountIssueRegressionBase):
    trigger_date_config = ("0 10 6 * * ", 6)
    check_date = _get_date_from_refresh_routing_config(trigger_date_config)

    def setUp(self):
        self.granularity = None
        self.is_routine_test = True
        self.ad_app_ids = random.sample(AD_APP_ID, 100)

    def test_pub_app_count_none(self):
        self.market_pub_app_count_none()

    def test_pub_app_count_not_none(self):
        self.market_pub_app_count_not_none()

    def test_pub_app_count_list_equal(self):
        self.market_pub_app_count_list_equal()

    def test_ad_platform_count_none(self):
        self.market_ad_platform_count_none()

    def test_ad_platform_count_not_none(self):
        self.market_ad_platform_count_not_none()

    def test_ad_platform_count_list_equal(self):
        self.market_ad_platform_count_list_equal()

    def test_not_all_ad_platform_count_list_none(self):
        self.market_not_all_ad_platform_count_list_none()

    def test_country_count_none(self):
        self.market_country_count_none()

    def test_country_count_not_none(self):
        self.market_country_count_not_none()

    def test_country_count_list_equal(self):
        self.market_country_count_list_equal()

    def test_country_count_list_not_ww(self):
        self.market_country_count_list_not_ww()

    def test_creative_count(self):
        self.market_creative_count_equal_advertiser_report_creative()


class TestMarketAllDateRangeRegression(TestMarketCountIssueRegressionBase):

    def setUp(self):
        self.granularity = None
        self.is_routine_test = False

    def test_pub_app_count_none(self):
        self.market_pub_app_count_none()

    def test_pub_app_count_not_none(self):
        self.market_pub_app_count_not_none()

    def test_pub_app_count_list_equal(self):
        self.market_pub_app_count_list_equal()

    def test_ad_platform_count_none(self):
        self.market_ad_platform_count_none()

    def test_ad_platform_count_not_none(self):
        self.market_ad_platform_count_not_none()

    def test_ad_platform_count_list_equal(self):
        self.market_ad_platform_count_list_equal()

    def test_not_all_ad_platform_count_list_none(self):
        self.market_not_all_ad_platform_count_list_none()

    def test_country_count_none(self):
        self.market_country_count_none()

    def test_country_count_not_none(self):
        self.market_country_count_not_none()

    def test_country_count_list_equal(self):
        self.market_country_count_list_equal()

    def test_country_count_list_not_ww(self):
        self.market_country_count_list_not_ww()
    
def debug(case_list):
    std_out_origin= sys.stdout
    std_err_origin= sys.stderr
    try:
        suite =  unittest.TestSuite()
        for case in case_list:
            suite.addTest(case)
        runner = unittest.TextTestRunner(verbosity=2, buffer=True)
        runner.run(suite)
    except Exception as ex:
        print dir(ex)
        print ex.message
        traceback.print_exception(type(ex), ex, ex.__traceback__)
    finally:
        sys.stdout = std_out_origin
        sys.stderr = std_err_origin
    
# case_name_list = [
#     "test_pub_count_none",
    
#     ]

test_case_list = [ ] # pass 
# test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketCountIssueMonthlyRegression))
# test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketCountIssueWeeklyRegression))
# test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketWeeklyCheck))
# test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketMonthlyCheck))
# test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketLogsFactAndSeenDaily))
# test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketDimensionDaily))

test_case_list.append(unittest.TestLoader().loadTestsFromTestCase(TestMarketAllDateRangeRegression))


debug(test_case_list)


In [0]:
%%sh
%%sh
PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF  
-- select creative_count,ad_platform_id from market.market_adapp_fact_v1 where granularity='weekly' and device_code = 'ios-phone' and  country_code = 'WW' and date='2020-03-28' and ad_app_id=585193266  and ad_platform_id=1;

-- select count(distinct(creative_id)) from  market.market_creative_adapp_fact_v1 where granularity='weekly' and  device_code = 'ios-phone' and country_code in ('AU', 'BR', 'CA', 'CN', 'DE', 'DK', 'ES', 'FI', 'FR', 'GB', 'HK', 'ID', 'IN', 'IT', 'JP', 'KR', 'MX', 'NO', 'NZ', 'RU', 'SE', 'TH', 'TR', 'TW', 'US') and date='2020-03-28' and ad_app_id=585193266;

select count(creative_id) from market.market_creative_adapp_fact_v1 where country_code = 'US' and device_code = 'ios-phone' and date='2020-04-30' and granularity = 'monthly' and ad_platform_id=1;



select count(1) from market.market_creative_adapp_fact_v1 where country_code = 'US' and device_code = 'ios-phone'  and granularity = 'monthly' and date='2020-04-30' and ad_platform_id = 1 and creative_id in (select distinct creative_id from market.market_creative_log_fact_v1 where country_code = 'US'  and device_code = 'ios-phone' and date BETWEEN '2020-04-01' and '2020-04-30' and ad_platform_id = 1);




select * from market.market_creative_map_dimension_v1 where update_date='2020-03-30' limit 10;

EOF


In [0]:

import unittest
import datetime
import croniter

from aadatapipelinecore.core.utils.commandline import env
from aadatapipelinecore.core.utils.encode import activate_system_utf8
from aadatapipelinecore.core.utils.spark import create_spark



def _get_check_date_from_routing_config(trigger_date_config):
        """
        return the date of : <days_delta> ago from previous scheduled date&time according to <cron_time>.
        e.g.
        config = ("0 9 * * *", 1), today is 2019-10-27 8:00
        so previous scheduled date&time is 2019-10-26 9:00
        will return "2019-10-25"

        e.g.
        config = ("0 7 * * *", 1), today is 2019-10-27 8:00
        so previous scheduled date&time is 2019-10-27 7:00
        will return "2019-10-26"

        Cron Time Format
        Character	Descriptor	        Acceptable values
        1	        Minute	            0 to 59, or * (no specific value)
        2	        Hour	            0 to 23, or * for any value. All times UTC.
        3	        Day of the month	1 to 31, or * (no specific value)
        4	        Month	            1 to 12, or * (no specific value)
        5	        Day of the week	    0 to 7 (0 and 7 both represent Sunday), or * (no specific value)

        :param trigger_datetime: the test trigger date
        :type trigger_datetime: object
        :return: date obj of "%Y-%m-%d"
        :type return: object
        """
        schedule, days_delta = trigger_date_config
        # here use UTC now
        cron = croniter.croniter(schedule, datetime.datetime.now())
        date = cron.get_prev(datetime.datetime) - datetime.timedelta(days=days_delta)
        return date
trigger_date_config = ("0 10 * * 0", 8)
print _get_check_date_from_routing_config(trigger_date_config)